In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
from utilities_functions.intermediate_layer_extraction import return_layer_input_output
from utilities_functions.ri_calculator import find_smallest_variation_to_change
import deepmatcher as dm
import torch

In [3]:
attributes =['Beer_Name','Brew_Factory_Name','Style','ABV']

In [4]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('../models/beer_hybrid.pth')
model.cuda()

MatchingModel(
  (attr_summarizers): ModuleMap(
    (Beer_Name): Hybrid(
      (word_contextualizer): RNN(
        (rnn_groups): ModuleList(
          (0): GRU(300, 150, batch_first=True, bidirectional=True)
        )
        (dropouts): ModuleList(
          (0): Dropout(p=0)
        )
        (bypass_networks): ModuleList(
          (0): None
        )
        (input_dropout): NoMeta(
          (module): Dropout(p=0)
        )
      )
      (word_comparator): Attention(
        (alignment_networks): ModuleList(
          (0): AlignmentNetwork(
            (transform): Transform(
              (transforms): ModuleList(
                (0): Linear(in_features=300, out_features=300, bias=True)
                (1): Linear(in_features=300, out_features=300, bias=True)
              )
              (bypass_networks): ModuleList(
                (0): Bypass(
                  (highway_gate): Linear(in_features=300, out_features=300, bias=True)
                )
                (1): Bypass(


In [5]:
neg_classifier_inputs,neg_classifier_outputs,neg_ids = return_layer_input_output('../Structured/Beer/'
                                                                         ,'test_negatives.csv',32,model,
                                                                        model.classifier)

In [6]:
pos_classifier_inputs,pos_classifier_outputs,pos_ids = return_layer_input_output('../Structured/Beer',
                                                                       'test_positives.csv',32,model,
                                                                       model.classifier)

In [7]:
negative_classifier_inputs = list(map(lambda x: x[0],neg_classifier_inputs))
positive_classifier_inputs = list(map(lambda x: x[0],pos_classifier_inputs))

In [12]:
attribute_length = int((300*3)/len(attributes))
classifier_length = int(attribute_length*len(attributes))

In [13]:
type(classifier_length)

int

In [14]:
current_sample = 0
#each column of this matrix is related to a specific attribute
negatives_ri_matrix = []
for batch in negative_classifier_inputs:
    for sample_index in range(len(batch)):
        current_sample_ris = []
        for j,attribute in enumerate(attributes):
            print('Processing sample {} with attribute {}'.format(current_sample,attribute))
            it,variation = find_smallest_variation_to_change(model.classifier,classifier_length=classifier_length,
                                                             attribute_length=attribute_length,input_matrix=batch,
                                                             vector_index=sample_index,attribute=j,class_to_reach=1)
            current_sample_ris.append(variation)
        negatives_ri_matrix.append(current_sample_ris)
        current_sample+=1

Processing sample 0 with attribute Beer_Name
Processing sample 0 with attribute Brew_Factory_Name
Processing sample 0 with attribute Style
Processing sample 0 with attribute ABV
Processing sample 1 with attribute Beer_Name
Processing sample 1 with attribute Brew_Factory_Name
Processing sample 1 with attribute Style
Processing sample 1 with attribute ABV
Processing sample 2 with attribute Beer_Name
Processing sample 2 with attribute Brew_Factory_Name
Processing sample 2 with attribute Style
Processing sample 2 with attribute ABV
Processing sample 3 with attribute Beer_Name
Processing sample 3 with attribute Brew_Factory_Name
Processing sample 3 with attribute Style
Processing sample 3 with attribute ABV
Processing sample 4 with attribute Beer_Name
Processing sample 4 with attribute Brew_Factory_Name
Processing sample 4 with attribute Style
Processing sample 4 with attribute ABV
Processing sample 5 with attribute Beer_Name
Processing sample 5 with attribute Brew_Factory_Name
Processing s

Processing sample 45 with attribute Brew_Factory_Name
Processing sample 45 with attribute Style
Processing sample 45 with attribute ABV
Processing sample 46 with attribute Beer_Name
Processing sample 46 with attribute Brew_Factory_Name
Processing sample 46 with attribute Style
Processing sample 46 with attribute ABV
Processing sample 47 with attribute Beer_Name
Processing sample 47 with attribute Brew_Factory_Name
Processing sample 47 with attribute Style
Processing sample 47 with attribute ABV
Processing sample 48 with attribute Beer_Name
Processing sample 48 with attribute Brew_Factory_Name
Processing sample 48 with attribute Style
Processing sample 48 with attribute ABV
Processing sample 49 with attribute Beer_Name
Processing sample 49 with attribute Brew_Factory_Name
Processing sample 49 with attribute Style
Processing sample 49 with attribute ABV
Processing sample 50 with attribute Beer_Name
Processing sample 50 with attribute Brew_Factory_Name
Processing sample 50 with attribute 

In [15]:
ri_norms_negative_samples = []
for ri_list in negatives_ri_matrix:
    ri_norms_negative_samples.append(list(map(lambda x:torch.norm(x).data[0],ri_list)))

In [16]:
negatives_variation_df = pd.DataFrame(data= ri_norms_negative_samples,columns=attributes)

In [17]:
negatives_variation_df.head(10)

,Beer_Name,Brew_Factory_Name,Style,ABV
0,1.813113,1.833648,2.434515,2.417495
1,4.055906,4.070273,4.739480,4.756083
2,5.578187,5.539420,6.207850,6.222769
3,0.509242,0.509242,0.509242,0.509242
4,4.790353,4.753148,5.405146,5.406320
5,1.082423,1.087765,1.111172,1.108775
6,4.763588,4.731594,5.379742,5.391893
7,1.811159,1.838941,2.462501,2.437738
8,5.541512,5.516161,6.144713,6.180171
9,6.496103,6.447077,7.160349,7.167564


In [18]:
current_sample = 0
#each column of this matrix is related to a specific attribute
positives_ri_matrix = []
for batch in positive_classifier_inputs:
    for sample_index in range(len(batch)):
        current_sample_ris = []
        for j,attribute in enumerate(attributes):
            print('Processing sample {} with attribute {}'.format(current_sample,attribute))
            it,variation = find_smallest_variation_to_change(model.classifier,classifier_length=classifier_length,
                                                             attribute_length=attribute_length,input_matrix=batch,
                                                             vector_index=sample_index,attribute=j,class_to_reach=0)
            current_sample_ris.append(variation)
        positives_ri_matrix.append(current_sample_ris)
        current_sample+=1

Processing sample 0 with attribute Beer_Name
Processing sample 0 with attribute Brew_Factory_Name
Processing sample 0 with attribute Style
Processing sample 0 with attribute ABV
Processing sample 1 with attribute Beer_Name
Processing sample 1 with attribute Brew_Factory_Name
Processing sample 1 with attribute Style
Processing sample 1 with attribute ABV
Processing sample 2 with attribute Beer_Name
Processing sample 2 with attribute Brew_Factory_Name
Processing sample 2 with attribute Style
Processing sample 2 with attribute ABV
Processing sample 3 with attribute Beer_Name
Processing sample 3 with attribute Brew_Factory_Name
Processing sample 3 with attribute Style
Processing sample 3 with attribute ABV
Processing sample 4 with attribute Beer_Name
Processing sample 4 with attribute Brew_Factory_Name
Processing sample 4 with attribute Style
Processing sample 4 with attribute ABV
Processing sample 5 with attribute Beer_Name
Processing sample 5 with attribute Brew_Factory_Name
Processing s

In [19]:
ri_norms_positives = []
for ri_list in positives_ri_matrix:
    ri_norms_positives.append(list(map(lambda x:torch.norm(x).data[0],ri_list)))

In [20]:
positives_variation_df = pd.DataFrame(data= ri_norms_positives,columns=attributes)

In [21]:
positives_variation_df.head(14)

,Beer_Name,Brew_Factory_Name,Style,ABV
0,4.684353,5.367664,7.818158,7.687635
1,1.169079,1.177719,1.726304,1.716746
2,0.565161,0.565161,1.074266,1.071117
3,3.765159,4.368979,6.053763,5.957832
4,0.000000,0.000000,0.000000,0.000000
5,3.604105,3.742848,5.847515,5.762258
6,0.000000,0.000000,0.000000,0.000000
7,8.816920,9.558426,15.584998,13.480430
8,2.057970,2.609487,3.384536,3.347701
9,0.572603,0.572603,1.087922,1.085208
